In [1]:
import numpy as np
import pandas as pd
import matplotlib.pylab as plt
import sklearn
from sklearn.impute import SimpleImputer
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import MinMaxScaler
from sklearn.ensemble import GradientBoostingRegressor

In [2]:
train_set = pd.read_csv("train.csv")
test_set = pd.read_csv("test.csv")
y_train = train_set.SalePrice
train_set = train_set.drop(["SalePrice"], axis=1)
train_set = train_set.set_index('Id')
test_set = test_set.set_index('Id')

In [3]:
firstmerge = pd.concat([train_set, test_set], axis=0)
data_dummies= pd.get_dummies(firstmerge)
train_set = data_dummies.loc[0:1460]
test_set = data_dummies.loc[1461:]

In [4]:
my_imputer = SimpleImputer()
train_set = my_imputer.fit_transform(train_set)
test_set = my_imputer.fit_transform(test_set)

scaler = MinMaxScaler()
scaler.fit(train_set)
train_scaled = scaler.transform(train_set)
test_scaled = scaler.transform(test_set)

In [5]:

param_grid = {'learning_rate': [0.1, 0.05, 0.02, 0.01],
'max_depth': [4, 6],
'min_samples_leaf': [3, 5, 9, 17],

}
est = GradientBoostingRegressor(n_estimators=500, random_state=33)
gs_cv = GridSearchCV(est, param_grid, n_jobs=4).fit(train_scaled, y_train)

/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/sklearn/model_selection/_split.py:1943: FutureWarning: You should specify a value for 'cv' instead of relying on the default value. The default value will change from 3 to 5 in version 0.22.
  warnings.warn(CV_WARNING, FutureWarning)


In [6]:
test_result = gs_cv.predict(test_scaled)
df= pd.DataFrame({'SalePrice': test_result})
df.index.name='Id'
df.index +=1461
df.to_csv('gridsearch.csv')